# Build graph from preprocessed file
With a tabulated count:san file

## Read in packages, functions, and file

In [1]:
import sys
import networkx as nx
import matplotlib as plt
import numpy as np
import re
import chess
import subprocess
import timeit
import pickle
import stockfish
from stockfish import Stockfish
import chess
import chess.engine
import warnings
warnings.filterwarnings('ignore')

engine = chess.engine.SimpleEngine.popen_uci("../stockfish-10-64")
stockfish = Stockfish("../stockfish-10-64")

In [2]:
def varWeighted(scores, counts):
    if (len(counts)<2):
        return None
    else:
        weightedMean = sum([a*b for a,b in zip(scores,counts)])/sum(counts)
        scoreDiffs = [(score - weightedMean)**2 for score in scores]
        V1 = sum(counts)
        V2 = sum([count**2 for count in counts])
        var = (V1/(V1**2-V2)) * sum([a*b for a,b in zip(counts,scoreDiffs)])
        return var
    
def get_score(san):
    board = chess.Board()
    for move in san:
        board.push_san(move)
    currFen = board.fen()
    score = engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE)
    if('#' in str(score['score'])):
        if('-' in str(score['score'])):
            return -39765
        else:
            return 39765
    else:
        score = int(str(score["score"]))
    return score

def san_to_fen(san):
    board = chess.Board()
    for move in san:
        board.push_san(move)
    currFen = board.fen()
    return currFen

def get_node_sd(node):
    scores = []
    counts = []
    try:
        for neighbor in list(g.neighbors(node)):
            scores.append(nx.get_node_attributes(g, 'score')[neighbor])
            counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][node])
        return node, np.sqrt(varWeighted(scores,counts))
    except:
        return node, None

In [3]:
game_file = "../tab-trunc.txt"

with open(game_file, "r") as file_in:
    mainlineList = []
    for line in file_in:
        mainlineList.append(line)

#extract each string of moves
stringList = []
for mainLine in mainlineList:
    stringList.append(mainLine.split())

# stringList.sort(key = len) 


In [4]:
print(len(stringList))

692287


## Build and pickle graph

In [5]:
cnt = 0
start = timeit.default_timer()

#initialize graph with root node
g = nx.DiGraph()
g.add_node('root', score = 0, movelist = 'root')

for s in stringList:
    whiteMove = 1
    if(cnt%10000 == 0):
        print("strings processed:", cnt, "TIME:", timeit.default_timer()-start)
    count = int(s[0])
    if(count >= 30):
        board = chess.Board()
        parentFen = 'root'
        for move in s[1:]:
            try:
                san = s[1:s.index(move)+1]
                board.push_san(move)
                currFen = board.fen()
                if(currFen not in g.nodes):
                    board = chess.Board(currFen)
                    if('#' not in move):
                        score = engine.analyse(board, chess.engine.Limit(time=0.05))
                        if('#' in str(score['score'])):
                            if('-' in str(score['score'])):
                                score =  -39765
                            else:
                                score =  39765
                        else:
                            score = int(str(score["score"])) * whiteMove
    #                         stockfish.set_fen_position(currFen)
    #                         score = stockfish.get_evaluation()['value']
                    else:
                        score = -39765 * whiteMove
                    g.add_node(currFen,
                               score = score,
                               count = count,
                               san = san,
                               movelistCount = {parentFen:count})
                    g.add_edge(parentFen, currFen)
                else:
                    if(parentFen in nx.get_node_attributes(g, 'movelistCount')[currFen]):
                        nx.get_node_attributes(g, 'movelistCount')[currFen][parentFen] += count #increase parent count
                    else:
                        nx.get_node_attributes(g, 'movelistCount')[currFen][parentFen] = count #add parent and count
                        g.add_edge(parentFen, currFen) #add edge from parentFen to currFen
                    g.nodes[currFen]['count'] = nx.get_node_attributes(g, 'count')[currFen] + count #increase count
    #                 whiteMove *= -1
                parentFen = currFen
            except Exception as e:
                print(e)
                print(sys.exc_info()[0])
                print("Move",s)
                stockfish.set_fen_position(currFen)
                print(stockfish.get_evaluation()['value'])
                break
        cnt+=1
    else:
        cnt+=1
            
stop = timeit.default_timer()
print('Time: ', stop - start)
print("totalNodes:",len(g))

strings processed: 0 TIME: 0.000495484999999185
strings processed: 10000 TIME: 110.945748224
strings processed: 20000 TIME: 174.593456613
strings processed: 30000 TIME: 244.631377693
strings processed: 40000 TIME: 306.09325846900003
strings processed: 50000 TIME: 395.651876414
strings processed: 60000 TIME: 479.548524067
strings processed: 70000 TIME: 570.949282106
strings processed: 80000 TIME: 670.884163004
strings processed: 90000 TIME: 774.484918954
strings processed: 100000 TIME: 879.490363812
strings processed: 110000 TIME: 1022.155896536
strings processed: 120000 TIME: 1174.064849527
strings processed: 130000 TIME: 1343.5592655120001
strings processed: 140000 TIME: 1523.58047337
strings processed: 150000 TIME: 1712.635962697
strings processed: 160000 TIME: 1916.924873939
strings processed: 170000 TIME: 2156.691122776
strings processed: 180000 TIME: 2398.105228171
strings processed: 190000 TIME: 2660.522026021
strings processed: 200000 TIME: 2951.587246862
strings processed: 2100

In [6]:
g.nodes()['rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2']

{'score': 71,
 'count': 86074,
 'san': ['d4', 'Nf6'],
 'movelistCount': {'rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1': 86074}}

In [7]:
#nx.write_gpickle(g,"test.gpickle")

In [9]:
g = nx.read_gpickle('690k_30.gpickle')

In [10]:
g.nodes()[san_to_fen(['c4', 'Nf6', 'Nf3'])]

{'score': 7,
 'count': 430,
 'san': ['c4', 'Nf6', 'Nf3'],
 'movelistCount': {'rnbqkb1r/pppppppp/5n2/8/2P5/8/PP1PPPPP/RNBQKBNR w KQkq - 1 2': 430}}

## Test variance calcuilations

In [11]:
start = timeit.default_timer()

board = chess.Board()
board.push_san('d4') #push the move to the board
board.push_san('Nf6') #push the move to the board
currFen = board.fen()
print(currFen)
scores = []
counts = []
for neighbor in list(g.neighbors(currFen)):
    scores.append(nx.get_node_attributes(g, 'score')[neighbor])
    counts.append(nx.get_node_attributes(g, 'count')[neighbor])
    
print("sd:",np.sqrt(varWeighted(scores, counts)))

stop = timeit.default_timer()
print('Time: ', stop - start) 

rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq - 1 2
sd: 36.453967228234106
Time:  2.1070754819957074


In [12]:
gamesDict = {"root" : ['e4', 'c5'],
"safe" : ['e4', 'c5', 'c3'],
"risky" : ['e4', 'c5', 'd4'],
"root2" : ['e4', 'e6', 'd4', 'd5'],
"popular2.0" : ['e4', 'e6', 'd4', 'd5', 'Nc3'],
"popular2.1" : ['e4', 'e6', 'd4', 'd5', 'Nd2'],
"safe2" : ['e4', 'e6', 'd4', 'd5', 'exd5'],
"risky2" : ['e4', 'e6', 'd4', 'd5', 'e5'],
"Ruy" : ['e4', 'e5','Nf3', 'Nc6','Bb5'],
"Vienna" : ['e4' ,'e5','d4','exd4','c3']}

for key, game in gamesDict.items():
    currFen = san_to_fen(game)
    scores = []
    counts = []
    try:
        for neighbor in list(g.neighbors(currFen)):
            scores.append(nx.get_node_attributes(g, 'score')[neighbor])
            counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][currFen])
        print('{} {} ||| SD :'.format(key, game), np.sqrt(varWeighted(scores,counts)), ", total Games:", sum(counts))
    except:
        break

    

root ['e4', 'c5'] ||| SD : 53.17421298817888 , total Games: 214412
safe ['e4', 'c5', 'c3'] ||| SD : 33.803211905697694 , total Games: 9288
risky ['e4', 'c5', 'd4'] ||| SD : 112.59554494204002 , total Games: 19407
root2 ['e4', 'e6', 'd4', 'd5'] ||| SD : 35.29555838233849 , total Games: 42146
popular2.0 ['e4', 'e6', 'd4', 'd5', 'Nc3'] ||| SD : 38.82435952658399 , total Games: 8875
popular2.1 ['e4', 'e6', 'd4', 'd5', 'Nd2'] ||| SD : 73.60617708334175 , total Games: 3144
safe2 ['e4', 'e6', 'd4', 'd5', 'exd5'] ||| SD : 59.07250112917537 , total Games: 11806
risky2 ['e4', 'e6', 'd4', 'd5', 'e5'] ||| SD : 78.55239442769992 , total Games: 20829
Ruy ['e4', 'e5', 'Nf3', 'Nc6', 'Bb5'] ||| SD : 19.630732457868167 , total Games: 42312
Vienna ['e4', 'e5', 'd4', 'exd4', 'c3'] ||| SD : 35.14133659917036 , total Games: 6424


In [13]:
sd_dict = {}
for node in g.nodes():
    n, val = get_node_sd(node)
    sd_dict.update({n: val})
#     dict.update(Iterable_Sequence of key:value)
#     sd_list.append(get_node_sd(node))

sd_dict = {key:val for key, val in sd_dict.items() if val != None}

maxSD = max(sd_dict.items(), key=lambda x : x[1])
print('Max value in Dict: ', maxSD[1])
print('Key With Max value in Dict: ', maxSD[0])

Max value in Dict:  28366.29563407954
Key With Max value in Dict:  r1bqkb1r/pppp1ppp/2n2n2/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR w KQkq - 4 4


In [14]:
{k: v for k, v in sorted(sd_dict.items(), key=lambda item: item[1], reverse = True)}

{'r1bqkb1r/pppp1ppp/2n2n2/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR w KQkq - 4 4': 28366.29563407954,
 'rnb1k1nr/pppp2pp/8/4NP2/7q/8/PPPP1KPP/RNBQ1B1R w kq - 1 6': 28305.484450897497,
 'rnbqkb1r/ppp2ppp/3p1n2/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR w KQkq - 2 4': 28248.208801621386,
 'rnb1k1nr/pppp1ppp/8/2b5/2B1Pp1q/8/PPPP2PP/RNBQ1KNR w kq - 4 5': 28188.10472522053,
 'rnb1k1nr/pppp1ppp/5q2/2b1p3/8/1P4P1/P1PPPPBP/RNBQK1NR w KQkq - 1 4': 28142.142784443404,
 'rnbqkbnr/pp3ppp/2pp4/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 1 4': 28130.82907594442,
 'rn1qkbnr/pppb1ppp/3p4/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 3 4': 28078.503174136615,
 'r1bqkb1r/ppp3pp/2n2p2/3Bp1N1/8/5Q2/PPPP1PPP/RNB1K2R b KQkq - 0 7': 28053.754436795083,
 'r1b1k2r/pppp1Npp/2n5/4p3/2B1n2q/8/PPPP2PP/RNBQ2KR w kq - 2 8': 28047.39047576441,
 'r1bqkbnr/pp1p1pp1/7p/2pQ4/2B1P3/8/PPP2PPP/RNB1K2R b KQkq - 1 7': 27987.28639936355,
 'rnbqkbnr/pppp1ppp/8/4p3/6P1/5P2/PPPPP2P/RNBQKBNR b KQkq - 0 2': 27879.099061842007,
 'rn1qkbnr/ppp2ppp/3p4/4p3/2BPP3/5Q2/

In [16]:
currFen =  'rnbqkbnr/p3pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR b KQkq - 1 6'
scores = []
counts = []

for neighbor in list(g.neighbors(currFen)):
    print(neighbor, g.nodes()[neighbor])
    scores.append(nx.get_node_attributes(g, 'score')[neighbor])
    counts.append(nx.get_node_attributes(g, 'movelistCount')[neighbor][currFen])
print('SD :', np.sqrt(varWeighted(scores,counts)), ", total Games:", sum(counts))


rnb1kbnr/p3pppp/8/qp6/2pP4/4PQ2/1P3PPP/RNB1KBNR w KQkq - 2 7 {'score': 2150, 'count': 113, 'san': ['d4', 'd5', 'c4', 'dxc4', 'e3', 'b5', 'a4', 'c6', 'axb5', 'cxb5', 'Qf3', 'Qa5+'], 'movelistCount': {'rnbqkbnr/p3pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR b KQkq - 1 6': 113}}
r1bqkbnr/p3pppp/2n5/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR w KQkq - 2 7 {'score': 462, 'count': 190, 'san': ['d4', 'd5', 'c4', 'dxc4', 'e3', 'b5', 'a4', 'c6', 'axb5', 'cxb5', 'Qf3', 'Nc6'], 'movelistCount': {'rnbqkbnr/p3pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR b KQkq - 1 6': 190}}
rnb1kbnr/p1q1pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR w KQkq - 2 7 {'score': 420, 'count': 34, 'san': ['d4', 'd5', 'c4', 'dxc4', 'e3', 'b5', 'a4', 'c6', 'axb5', 'cxb5', 'Qf3', 'Qc7'], 'movelistCount': {'rnbqkbnr/p3pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR b KQkq - 1 6': 34}}
rn1qkbnr/pb2pppp/8/1p6/2pP4/4PQ2/1P3PPP/RNB1KBNR w KQkq - 2 7 {'score': 655, 'count': 63, 'san': ['d4', 'd5', 'c4', 'dxc4', 'e3', 'b5', 'a4', 'c6', 'axb5', 'cxb5', 'Qf3', 'Bb7'], 'movelistCount':

## Ad hoc viz of moves and children with sf score

In [ ]:
#Move ['32', 'f3', 'e5', 'g4', 'Qh4#']

board = chess.Board()
board.push_san('f3') #push the move to the board
board.push_san('e5') #push the move to the board
#board.push_san('g4') #push the move to the board
#board.push_san('Qh4#') #push the move to the board
currFen = board.fen()
print(currFen)
stockfish.set_fen_position(currFen)
score = stockfish.get_evaluation()['value']
print(score)

In [ ]:
board = chess.Board(currFen)
print(engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE))
board

In [ ]:
board = chess.Board(currFen)
print(engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE))
print("CURRFEN:")
print(board)
print("_"*50)
print("_"*50)
for neighbor in list(g.neighbors(currFen)):
    board = chess.Board(neighbor)
    print(engine.analyse(board, chess.engine.Limit(time=.05), info=chess.engine.INFO_SCORE))
    print(board)
    print("_"*50)